# Completion Times Cleaning

## Cleaning
1. Check for any recording that has 7 or > subtask topics. Check in the questionnaire notes what was the reason for the typo.
2. Check in the questionnaire notes for any additional corrections that should be done to the completion times
3. Correct subatask topic files based on conclusions
4. Checking missing data (subtasks not recorded?)

## 1. Clean Tasks 7 or >

In [43]:
import os
import pandas as pd

# Base path to the 'data' folder
base_path = r"E:\Hydra\sophia_ws\SerifosUpload\USER_STUDY\data"

# Initialize an empty list to store all data rows
rows = []

# Walk through the folder structure
for root, dirs, files in os.walk(base_path):
    for file in files:
        if file == "subtask_topic.csv":
            # Full path to the CSV file
            file_path = os.path.join(root, file)
            
            # Get the parent folder name that ends with "-ROSBAG"
            rosbag_folder = os.path.basename(os.path.dirname(file_path))
            
            # Determine if the file is from the "CHIRON" or "WBC" folder
            study_type = "CHIRON" if "CHIRON" in file_path else "WBC" if "WBC" in file_path else "UNKNOWN"
            
            # Determine if the folder after CHIRON/WBC is "WITH-VR" or "NO-VR"
            folder_parts = file_path.split(os.sep)
            vr_type = "UNKNOWN"
            if study_type in folder_parts:
                study_index = folder_parts.index(study_type)
                if study_index + 1 < len(folder_parts):
                    vr_type = folder_parts[study_index + 1]
            
            # Read the CSV file
            csv_data = pd.read_csv(file_path)
            
            # Extract the data column, converting it into individual columns for the DataFrame
            data_columns = csv_data['data'].tolist()  # Assuming column 'data' exists in the CSV
            
            # Create a row for this CSV
            row_dict = {'Folder': rosbag_folder, 'Controller': study_type, 'Start_Modality': vr_type}
            row_dict.update({f'Task_{i+1}': value for i, value in enumerate(data_columns)})
            
            # Append the row dictionary to the list
            rows.append(row_dict)

# Convert the list of dictionaries into a DataFrame
result_df = pd.DataFrame(rows)

In [4]:
result_df

Folder Controller Start_Modality      Task_1  \
0    AAHF21-RW-WITH-VR-TRIAL-2-ROSBAG     CHIRON        WITH-VR   75.644989   
1    AAHF21-RW-WITH-VR-TRIAL-1-ROSBAG     CHIRON        WITH-VR  165.343307   
2      AAHF21-RW-NO-VR-TRIAL-2-ROSBAG     CHIRON        WITH-VR   37.954384   
3      AAHF21-RW-NO-VR-TRIAL-1-ROSBAG     CHIRON        WITH-VR   26.210884   
4      AAHF21-RW-NO-VR-TRIAL-3-ROSBAG     CHIRON        WITH-VR   38.964264   
..                                ...        ...            ...         ...   
105    NRKH02-RW-NO-VR-TRIAL-3-ROSBAG        WBC        WITH-VR   92.424736   
106  NRKH02-RW-WITH-VR-TRIAL-3-ROSBAG        WBC        WITH-VR  125.898254   
107  NRKH02-RW-WITH-VR-TRIAL-1-ROSBAG        WBC        WITH-VR  130.299973   
108    NRKH02-RW-NO-VR-TRIAL-1-ROSBAG        WBC        WITH-VR   94.716003   
109  NRKH02-RW-WITH-VR-TRIAL-2-ROSBAG        WBC        WITH-VR  179.640030   

         Task_2      Task_3      Task_4      Task_5      Task_6  Task_7  
0    124.517296  163.717560  181.565674  212.469955         NaN     NaN  
1    472.945618  507.929932  535.194153  573.010437         NaN     NaN  
2     82.738716  101.738884  119.715019  139.955200         NaN     NaN  
3     54.835251   76.899284   90.275398  108.467636         NaN     NaN  
4     80.348549   97.684708  110.356728  126.581322         NaN     NaN  
..          ...         ...         ...         ...         ...     ...  
105  173.769287  175.753204  190.041260  210.689484  249.169754     NaN  
106  187.962341  297.587280  371.875946  457.940491  509.476929     NaN  
107  178.796204  219.332550  283.437073  638.647339  705.255676     NaN  
108  152.324249  238.612961  271.973236  335.173828  364.790039     NaN  
109  228.016525  317.105133  368.977570  433.386597  509.170685     NaN  

[110 rows x 10 columns]

In [5]:
# Checking for rows that contain 7 subtask topics
filtered_rows = result_df[result_df["Task_7"].notna()]
filtered_rows

Folder Controller Start_Modality      Task_1  \
59    ARAH22-RW-NO-VR-TRIAL-3-ROSBAG        WBC          NO-VR   61.152534   
60    ARAH22-RW-NO-VR-TRIAL-2-ROSBAG        WBC          NO-VR   92.701889   
62  ARAH22-RW-WITH-VR-TRIAL-1-ROSBAG        WBC          NO-VR   57.901505   
72  ELBH19-RW-WITH-VR-TRIAL-2-ROSBAG        WBC          NO-VR  175.885376   
95    IICM04-RW-NO-VR-TRIAL-3-ROSBAG        WBC        WITH-VR  213.295090   

        Task_2      Task_3      Task_4      Task_5      Task_6      Task_7  
59  109.153099  113.704910  141.705109  150.673386  171.041397  190.953384  
60  127.758286  168.430267  199.422531  215.262604  274.423065  294.503204  
62   93.733719  162.918350  199.886688  203.374664  253.879059  280.311279  
72  306.989960  339.446136  561.863403  688.344116  897.361145  949.409485  
95  278.391144  380.303619  431.447968  534.456665  604.792847  605.776917

Looking at the notes:
- IICM04 NO VR TRIAL 3: Task_6 misclicked, remove row
- ELBH19 EITH VR TRIAL 2: Task_1 misclicked, remove row
- ARAH22: Was trying to capture movement to and from counter, ended up not needing it + misclicked to close drawer

In [41]:
# Function to highlight specified cells based on the index
def highlight_specific_cells(x):
    # Create empty style DataFrame with the same shape as the original dataframe
    styles = pd.DataFrame('', index=x.index, columns=x.columns)
    styles.loc[59, 'Task_2'] = 'background-color: yellow;'
    styles.loc[60, 'Task_4'] = 'background-color: yellow;'
    styles.loc[62, 'Task_1'] = 'background-color: yellow;'
    styles.loc[72, 'Task_1'] = 'background-color: yellow;'
    styles.loc[95, 'Task_2'] = 'background-color: yellow;'
    
    # Return the styling DataFrame
    return styles

# Apply the function to style the DataFrame
styled_df = filtered_rows.style.apply(highlight_specific_cells, axis=None)

# Display the styled DataFrame in Jupyter
styled_df

I will manually fix that in the dataset to exclude those additional completion times

## 2. Clean 4 Tasks or <

In [31]:
import os
import pandas as pd

# Base path to the 'data' folder
base_path = r"E:\Hydra\sophia_ws\SerifosUpload\USER_STUDY\data"

# Initialize an empty list to store all data rows
rows = []

# Walk through the folder structure
for root, dirs, files in os.walk(base_path):
    for file in files:
        if file == "subtask_topic.csv":
            # Full path to the CSV file
            file_path = os.path.join(root, file)
            
            # Get the parent folder name that ends with "-ROSBAG"
            rosbag_folder = os.path.basename(os.path.dirname(file_path))
            
            # Determine if the file is from the "CHIRON" or "WBC" folder
            study_type = "CHIRON" if "CHIRON" in file_path else "WBC" if "WBC" in file_path else "UNKNOWN"
            
            # Determine if the folder after CHIRON/WBC is "WITH-VR" or "NO-VR"
            folder_parts = file_path.split(os.sep)
            vr_type = "UNKNOWN"
            if study_type in folder_parts:
                study_index = folder_parts.index(study_type)
                if study_index + 1 < len(folder_parts):
                    vr_type = folder_parts[study_index + 1]
            
            # Read the CSV file
            csv_data = pd.read_csv(file_path)
            
            # Extract the data column, converting it into individual columns for the DataFrame
            data_columns = csv_data['data'].tolist()  # Assuming column 'data' exists in the CSV
            
            # Create a row for this CSV
            row_dict = {'Folder': rosbag_folder, 'Controller': study_type, 'Start_Modality': vr_type}
            row_dict.update({f'Task_{i+1}': value for i, value in enumerate(data_columns)})
            
            # Append the row dictionary to the list
            rows.append(row_dict)

# Convert the list of dictionaries into a DataFrame
df = pd.DataFrame(rows)

In [20]:
# Check for NaN in all columns except Task_6 and Task_7
nan_check = df.loc[:, df.columns.difference(['Task_6', 'Task_7'])].isna().any(axis=1)

# Filter rows with NaN in these columns
rows_with_nan = df[nan_check]

# Display the rows
rows_with_nan

# Function to highlight specified cells based on the index
def highlight_specific_cells(x):
    # Create empty style DataFrame with the same shape as the original dataframe
    styles = pd.DataFrame('', index=x.index, columns=x.columns)
    
    # Highlight Task_4 for the first 3 rows using the actual indices 59, 60, and 62
    styles.loc[25] = 'background-color: yellow;'
    styles.loc[65] = 'background-color: yellow;'

    # Return the styling DataFrame
    return styles

# Apply the function to style the DataFrame
styled_df = rows_with_nan.style.apply(highlight_specific_cells, axis=None)

# Display the styled DataFrame in Jupyter
styled_df

3 of them are from the simulation experiment, which was not expected to include all tasks anyways. For the last trial of EECW29, she had some problems opening the drawer. The recording ends at 183.51. Looking at the other trials, I probably forgot to set first subtask for the thirds trial without VR:

In [15]:
# Filter rows where 'Folder' contains "EECW"
eecw_rows = df[df['Folder'].str.contains("EECW", na=False)]

# Display the rows
eecw_rows

# Function to highlight specified cells based on the index
def highlight_specific_cells(x):
    # Create empty style DataFrame with the same shape as the original dataframe
    styles = pd.DataFrame('', index=x.index, columns=x.columns)
    
    # Highlight Task_4 for the first 3 rows using the actual indices 59, 60, and 62
    styles.loc[25] = 'background-color: yellow;'

    # Return the styling DataFrame
    return styles

# Apply the function to style the DataFrame
styled_df = eecw_rows.style.apply(highlight_specific_cells, axis=None)

# Display the styled DataFrame in Jupyter
styled_df

So we need to replace the values of `Task_1`and `Task_4`, that were no properly recorded. Since in the notes it is written she had some problems opening the drawer, will give more time for that subtask and take the mean of the others for the fourth task. New data for `EECW29-RW-NO-VR-TRIAL-3-ROSBAG	`:

|Time|data|
|--|--|
|1734086524.1244333|68|
|1734086542.1089532|85.9845199584961|
|1734086585.069385|128.94493103027344|
|1734086613.124454|157|
|1734086622.3257344|166.2012939453125|
|1734086639.6378925|183.51341247558594|

Now about the `BTHH23-RW-NO-VR-TRIAL-1-ROSBAG`

In [32]:
# Filter rows where 'Folder' contains "EECW"
bthh_rows = df[df['Folder'].str.contains("BTHH", na=False)]

# Display the rows
bthh_rows

# Function to highlight specified cells based on the index
def highlight_specific_cells(x):
    # Create empty style DataFrame with the same shape as the original dataframe
    styles = pd.DataFrame('', index=x.index, columns=x.columns)
    
    # Highlight Task_4 for the first 3 rows using the actual indices 59, 60, and 62
    styles.loc[65] = 'background-color: yellow;'

    # Return the styling DataFrame
    return styles

# Apply the function to style the DataFrame
styled_df = bthh_rows.style.apply(highlight_specific_cells, axis=None)

# Display the styled DataFrame in Jupyter
styled_df

The time for the first two tasks were skipped, let's set a reasonable value for that no to lose the data.

|Time|data|
|--|--|
|1732818932.4626224|160|
|1732819014.1471317|241.68450927734375|
|1732819182.4625947|410|
|1732819193.5882049|421.1256103515625|
|1732819247.0124893|474.5499267578125|
|1732819284.38901|511.9264221191406|


## 3. Correct times based on notes

- ATSF08: "Third trial with VR consider 234 for closing drawer not 215" - OK
- ELBH19: "In the 3rd task with VR had a hard time opening drawer because of gripper opening, rm 1min52 minutes before the first time collected" - OK
- RENL05: "At the second trial with VR, stopped for 10 seconds to adjust headset" - Reduced 10 seconds from trial

## 4. Check for double clicks that are less than 10 seconds apart

In [50]:
import os
import pandas as pd

# Base path to the 'data' folder
base_path = r"E:\Hydra\sophia_ws\SerifosUpload\USER_STUDY\data"

# Initialize an empty list to store all data rows
rows = []

# Walk through the folder structure
for root, dirs, files in os.walk(base_path):
    for file in files:
        if file == "subtask_topic.csv":
            # Full path to the CSV file
            file_path = os.path.join(root, file)
            
            # Get the parent folder name that ends with "-ROSBAG"
            rosbag_folder = os.path.basename(os.path.dirname(file_path))
            
            # Determine if the file is from the "CHIRON" or "WBC" folder
            study_type = "CHIRON" if "CHIRON" in file_path else "WBC" if "WBC" in file_path else "UNKNOWN"
            
            # Determine if the folder after CHIRON/WBC is "WITH-VR" or "NO-VR"
            folder_parts = file_path.split(os.sep)
            vr_type = "UNKNOWN"
            if study_type in folder_parts:
                study_index = folder_parts.index(study_type)
                if study_index + 1 < len(folder_parts):
                    vr_type = folder_parts[study_index + 1]
            
            # Read the CSV file
            csv_data = pd.read_csv(file_path)
            
            # Extract the data column, converting it into individual columns for the DataFrame
            data_columns = csv_data['data'].tolist()  # Assuming column 'data' exists in the CSV
            
            # Create a row for this CSV
            row_dict = {'Folder': rosbag_folder, 'Controller': study_type, 'Start_Modality': vr_type}
            row_dict.update({f'Task_{i+1}': value for i, value in enumerate(data_columns)})
            
            # Append the row dictionary to the list
            rows.append(row_dict)

# Convert the list of dictionaries into a DataFrame
df = pd.DataFrame(rows)

In [52]:
import pandas as pd
import numpy as np

# Create a subset DataFrame that only contains rows with red-highlighted cells
def get_red_highlighted_rows(data):
    # Create an empty list to store rows and corresponding styles
    rows_with_red = []
    styles = []

    # Iterate through each row in the dataset
    for row_index, row in data.iterrows():
        # Convert row to a NumPy array of float values for numerical operations
        numerical_values = row.drop(['Folder', 'Controller', 'Start_Modality'], errors='ignore').values

        # Check if any adjacent values are less than 8 seconds apart
        red_row = False
        row_style = [''] * len(row)
        for col_idx in range(len(numerical_values) - 1):
            if np.abs(numerical_values[col_idx] - numerical_values[col_idx + 1]) < 8:
                # Mark both adjacent cells
                red_row = True
                row_style[col_idx + 3] = 'background-color: red;'  # Offset +3 for non-task columns
                row_style[col_idx + 4] = 'background-color: red;'

        # Append the row and style if it contains red cells
        if red_row:
            rows_with_red.append(row)
            styles.append(row_style)

    # Create DataFrame from the filtered rows and styles
    filtered_df = pd.DataFrame(rows_with_red, columns=data.columns)
    style_df = pd.DataFrame(styles, index=filtered_df.index, columns=filtered_df.columns)
    return filtered_df, style_df

# Extract the subset and its styles
filtered_rows, highlight_styles = get_red_highlighted_rows(df)

# Apply styles to the filtered DataFrame
styled_filtered_df = filtered_rows.style.apply(lambda x: highlight_styles.loc[x.name], axis=1)

# Display the highlighted DataFrame in Jupyter Notebook
styled_filtered_df

This happened mostly when getting to the bottle and picking it up was very fast. Some of these cases I forgot to set the time that the robot had arrived in front of the bottle and double clicked later. So I will remove the ones I have written down to have done that. Some of them are actually true though, some people were able to get to the bottle and immediately pick it up. The ones I will remove one of the columns:

`NRKH02-RW-NO-VR-TRIAL-3-ROSBAG`
`EECW29-RW-NO-VR-TRIAL-1-ROSBAG`

## 4. Checking missing data

In [62]:
# Filter out rows with 'SIM' in the 'Folder' column and explicitly create a copy
filtered_df = df[~df['Folder'].str.contains("SIM", na=False)].copy()

# Extract the first 6 characters (prefix) from the 'Folder' column
filtered_df['Prefix'] = filtered_df['Folder'].str[:6]

# Count the number of rows for each prefix
prefix_counts_without_sim = filtered_df['Prefix'].value_counts()

# Display the counts
print(prefix_counts_without_sim)

Prefix
AAHF21    6
ATSF08    6
MGNH15    6
IICM04    6
DOFS27    6
SASD12    6
ELBH19    6
BTHH23    6
ARAH22    6
NULS22    6
CEEJ05    6
HASJ22    6
EECW29    6
RENL05    6
DAMA12    6
NRKH02    6
DOCS21    5
XHKB15    3
REKD03    2
LEMT02    1
Name: count, dtype: int64


|Folder|Count|What is missing?|
|--|--|--|
|DOCS21|5|NO-VR-TRIAL-1|
|XHKB15|3|WITH-VR all|
|REKD03|2|NO-VR trial 3 and WITH-VR all|
|LEMT02|1|NO-VR all and WITH-VR trials 2 and 3|
